# CoVID19 Findings

This file is part of a University Project on Data Mining and Machine Learning.
Brought to you by Ioannis Loudaros and Christina Kratimenou,students in the University of Patras.
For more information read the report of the Project [here](https://github.com/iloudaros/Data-Mining-and-Machine-Learning-Project)

In [3]:
# data visualization
import seaborn as sns
%matplotlib inline
from matplotlib import pyplot as plt
import pandas as pd

#Importing the dataset
dataset_file = "given/data.csv"

dataset = pd.read_csv(dataset_file)

#Exploring the dataset
dataset.info()
dataset.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38472 entries, 0 to 38471
Data columns (total 15 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Entity                           38472 non-null  object 
 1   Continent                        38472 non-null  object 
 2   Latitude                         38472 non-null  float64
 3   Longitude                        38472 non-null  float64
 4   Average temperature per year     38472 non-null  int64  
 5   Hospital beds per 1000 people    38472 non-null  float64
 6   Medical doctors per 1000 people  38472 non-null  float64
 7   GDP/Capita                       38472 non-null  float64
 8   Population                       38472 non-null  int64  
 9   Median age                       38472 non-null  int64  
 10  Population aged 65 and over (%)  38472 non-null  int64  
 11  Date                             38472 non-null  object 
 12  Daily tests       

,Latitude,Longitude,Average temperature per year,Hospital beds per 1000 people,Medical doctors per 1000 people,GDP/Capita,Population,Median age,Population aged 65 and over (%),Daily tests,Cases,Deaths
count,38472.000000,38472.000000,38472.000000,38472.000000,38472.000000,38472.000000,3.847200e+04,38472.000000,38472.000000,3.057700e+04,3.821800e+04,34862.000000
mean,23.737060,20.210847,17.717145,3.165305,2.086313,19002.331415,4.896983e+07,32.751508,10.663392,3.944059e+04,2.879027e+05,8090.504733
std,26.060413,61.074922,8.134926,2.561767,1.518252,22271.107786,1.427251e+08,8.472305,6.770417,1.501847e+05,1.405243e+06,29548.753221
min,-40.900000,-106.350000,-2.000000,0.200000,0.020000,411.600000,3.412840e+05,16.000000,1.000000,-2.391720e+05,1.000000e+00,1.000000
25%,8.620000,-3.440000,11.000000,1.400000,0.820000,3659.000000,4.793900e+06,27.000000,5.000000,1.505000e+03,2.074000e+03,77.000000
50%,27.510000,21.820000,20.000000,2.500000,1.890000,8821.800000,1.148464e+07,32.000000,8.000000,5.520000e+03,2.143100e+04,527.000000
75%,45.940000,47.480000,25.000000,4.490000,3.210000,25946.200000,4.286296e+07,41.000000,16.000000,2.038200e+04,1.373770e+05,3480.500000
max,64.960000,179.410000,29.000000,13.050000,7.520000,114704.600000,1.339180e+09,48.000000,28.000000,2.945871e+06,2.860567e+07,513091.000000


In [4]:
print(dataset.head())

    Entity Continent  Latitude  Longitude  Average temperature per year  \
0  Albania    Europe     41.15      20.17                            14   
1  Albania    Europe     41.15      20.17                            14   
2  Albania    Europe     41.15      20.17                            14   
3  Albania    Europe     41.15      20.17                            14   
4  Albania    Europe     41.15      20.17                            14   

   Hospital beds per 1000 people  Medical doctors per 1000 people  GDP/Capita  \
0                           2.89                             1.29      5353.2   
1                           2.89                             1.29      5353.2   
2                           2.89                             1.29      5353.2   
3                           2.89                             1.29      5353.2   
4                           2.89                             1.29      5353.2   

   Population  Median age  Population aged 65 and over (%)    

# Data Preproccessing


## B. Data Transformation

# Correlations
A good source of titanic dataset statistical analysis starting with the basics and concludes on analysing various correlations among the dataset at the following link [^1]

[^1]: https://makeschool.org/mediabook/oa/tutorials/titanic-dataset-tutorial-an-intro-to-data-analysis-and-statistics-n40/jupyter-notebooks/

### Graphical Correlations

### Correlation Heatmap

# Machine Learning Models

### Visualization